# Spark-Matcher example 

This notebook shows how to use the `spark_matcher`. First we create a Spark session:

In [ ]:
%config Completer.use_jedi = False  # for proper autocompletion
from pyspark.sql import SparkSession

In [ ]:
spark = (SparkSession
             .builder
             .master("local")
             .enableHiveSupport()
             .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/24 13:44:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In this notebook we use some example data that comes with `spark_matcher`

In [ ]:
from spark_matcher.data import load_data

In [ ]:
a, b = load_data(spark)

The two dataframes `a` and `b` both contain records North Carolina Voter Registry data. For each person there is a name, suburb and postcode:

In [ ]:
a.limit(2).toPandas()

,name,suburb,postcode
0,kiera matthews,charlotte,28216
1,khimerc thomas,charlotte,2826g


In [ ]:
b.limit(2).toPandas()

,name,suburb,postcode
0,kiea matthews,charlotte,28218
1,chimerc thmas,chaflotte,28269


We use the `spark_matcher` to link the records in dataframe `a` with the records in dataframe `b`. First import the `Matcher` and create an instance. The fields that are used for matching are given as the `col_names` argument:

In [ ]:
from spark_matcher.matcher import Matcher

In [ ]:
myMatcher = Matcher(spark, col_names=['name', 'suburb', 'postcode'])

Now we are ready for fitting the `Matcher` object using 'active learning'; this means that the user has to enter whether a pair is a match or not. 
You enter 'y' if a pair is a match or 'n' when a pair is not a match. You will be notified when the model has converged and you can stop training by pressing 'f'.

In [ ]:
myMatcher.fit(a, b)


Nr. 1 (0+/0-)
Is this a match? (y)es, (n)o, (p)revious, (s)kip, (f)inish

name_1: kiea matthews
suburb_1: charlotte
postcode_1: 28218

name_2: kiea matthews
suburb_2: charlotte
postcode_2: 28218



 y



Nr. 2 (1+/0-)
Is this a match? (y)es, (n)o, (p)revious, (s)kip, (f)inish

name_1: khimerc thomas
suburb_1: charlotte
postcode_1: 2826g

name_2: kiea matthews
suburb_2: charlotte
postcode_2: 28218



 n



Nr. 3 (1+/1-)
Is this a match? (y)es, (n)o, (p)revious, (s)kip, (f)inish

name_1: john bentzen
suburb_1: waxhaw
postcode_1: 28173

name_2: john hanegraaff
suburb_2: waxhaw
postcode_2: 28173



...

 y


Classifier converged, enter 'f' to stop training

Nr. 12 (6+/5-)
Is this a match? (y)es, (n)o, (p)revious, (s)kip, (f)inish

name_1: de1>ra benf:eld
suburb_1: concord
postcode_1: 28025

name_2: debra benfield
suburb_2: concord
postcode_2: 28025



 y


Classifier converged, enter 'f' to stop training

Nr. 13 (7+/5-)
Is this a match? (y)es, (n)o, (p)revious, (s)kip, (f)inish

name_1: ruth edwards
suburb_1: henderson
postcode_1: 27536

name_2: raechaun edwards
suburb_2: lillington
postcode_2: 27546



 f


The `Matcher` is now trained and can be used to predict on all data. This can be the data used for training or new data that was not seen by the model yet.
By default the `threshold` is 0.5. A lower threshold results in more matches but also in more incorrect matches. By setting the `top_n` you can choose how 
many matches you want to see if there are more than one matche for a particular record.

In [ ]:
result = myMatcher.predict(a, b, threshold=0.5, top_n=3)

Now let's have a look at the results:

In [ ]:
result_pdf = result.toPandas()

In [ ]:
result_pdf.sort_values('score')

,name_1,suburb_1,postcode_1,name_2,suburb_2,postcode_2,score
1011,teresina fontana,newport,28570,teres'lna fontana,newport,28571,0.802804
721,melissa wa5d,greensboro,274|0,melissa ward,greensboro,27410,0.954172
1026,thomas dy5on,statesville,z8677,thomas dyson,statesvile,28697,0.958674
512,judirh coile,charlotte,2821q,judith coile,charlott,28224,0.959523
340,ge0ffrey ryan,wilmington,z8403,geoffrey ryan,wilmnigton,28400,0.962752
...,...,...,...,...,...,...,...
368,helem farmer,lincolnton,2809z,helen farmer,lincolnton,28092,1.000000
367,heather stewart,concord,28027,heather stewart,concord,28027,1.000000
366,heather caywood,charlotte,28278,heather cayw0od,char1otte,28278,1.000000
372,henry wyatt,waynesville,28786,henty wyatt,waynesvillte,28786,1.000000


If you want to use the `Matcher` later without having to retrain, you can save the `Matcher` and load it later:

In [ ]:
myMatcher.save('myMatcher.pkl')

In [ ]:
myRestoredMatcher = Matcher(spark)

/Users/zv41nn/Documents/P00303-spark_matcher/spark_matcher/matcher/matcher.py:61: UserWarning: Either `col_names` or `field_info` should be provided. This instance can only be used when loading a previously saved instance.
  'Either `col_names` or `field_info` should be provided. This instance can only be used when loading a '


In [ ]:
myRestoredMatcher.load('myMatcher.pkl')

This `Matcher` object can be used to predict on new data.